In [1]:
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings('ignore')
import scipy.stats as stats
import scipy
import pandas as pd

<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>

# 挖掘频率

**频数模式（Frequent Pattern）**：是一种Pattern频繁的出现在一个数据集中。Frequent pattens are patterns that appear frequently in a data set.

比如交易数据中：一个人买了牛奶和面包，而这个牛奶和面包的集合频繁的出现在用户交易数据中。可称为**频数项集（Frequent itemset）**。如果一个
人首先购买了大米，然后购买了牛奶，最后购买了面包，那这个先后顺序就可能称为一个**序列模式**。

举个例子：

**Market basket analysis.** Suppose, as manager of an AllElectronics branch, you would like to learn more about the buying habits of your customers. Specifically, you wonder,“Which groups or sets of items are customers likely to purchase on a given trip to the store?”

To answer your question, market basket analysis may be performed on the retail data ofcustomer transactions at your store. You can then use the results to **plan marketing oradvertising strategies, or in the design of a new catalog**. For instance, market basket analysis may help you design different store layouts. In one strategy, items that are frequentl purchased together can be placed in proximity to further encourage the combined sale of such items. If customers who purchase computers also tend to buy antivirus software at the same time, then placing the hardware display close to the software display may help increase the sales of both items.

如果每种商品都有一个布尔变量，表示改商品是否出现在购物篮中。那马每一个购物篮可以使用一个布尔向量表示。分析布尔向量，得到反应商品**频数关联**或者同时购买的**购买模式**，而模式可以用**关联规则（Association Rule）**表示。比如：购买计算机也趋向于购买杀毒软件，可用下列关联规则表示：
$$ computer \Rightarrow software [support = 2\%; confidence = 60\%]  $$

一个是**支持度（Support）**，一个是**置信度（Confidence）**是规则兴趣度的两种度量。
* 支持度为 2%表示，所有情况中，只有2%的情况是发生了计算机和软件同时购买的情况。
* 置信度60%表示，购买了计算机的用户，60%也购买了软件。

如果一组数据同时满足 **最小支持度阀值**和 **最小置信度阀值**，则可以认为这个关联规则可能是有用的。

## 频率项集、闭项集和关联规则
设$I = {i_1,i_2,\cdots,i_m}$是项的集合。设任务相关的数据$D$是数据库的集合，其中每个事务$T$是一个非空项集，使得$T \subseteq I$。每一个事务都有一个标识符，称为$TID$。设A是一个项集，事务$T$包含$A$，当且仅当 $A \subseteq T$。关联规则是形如$A \Rightarrow B$的式子。其中$A \ subset I , B \subset I, A \ne \emptyset,B \ne \emptyset, A \cap B = \emptyset $.
     $$support(A \Rightarrow B) = P(A \cup B) $$
     $$confidence(A \Rightarrow B) = P(A | B) $$
同时满足最小支持度阀值（**min_sup**）和最小置信度阀值（**min_conf**）的关联规则称为**强规则**。

项的集合称为**项集**。包含k个项的项集称为k项集。项集出现的频度是包好你项集的事务数。

$$confidence(A\Rightarrow B) = P(B|A) = \frac{support(A \cup B)}{support(A)} = \frac{support\_count(A \cup B)}{support\_count(A)}$$

也就是说，只要能得$A，B，A \cup B$的支持度技术，就能够到处对应的关联规则：$A \Rightarrow B 和 B \Rightarrow A$

一般来说关联规则挖掘分为两个过程：
* 找出所有的频数项集：也就是招数所有频数大于min_sup的项
* 由频率项集产生强关联规则：这些规则必须满足min_sup, min_conf

项集X在数据集D上是**闭的**，如果不存在真超项集Y($X \subset Y$)，使得Y与X在D中具有相同的计数。项集X是数据集D中的**闭频数项集**，如果X在D中是闭的和频繁的。若只满足是频繁的，而不满足闭的，则称X是D中的**极大频繁项集，极大项集**。

举个例子：transaction表示事务
<img src="images/例子1.png" style="zoom:70%">

## Apriori（先验）算法
使用布尔关联规则挖掘频繁项集的算法。扫描数据库，累计每个项的计数，并收集满足最小支持度$min_sup$的项。

找出频率为$1$项集的集合，记为$L_1$，使用$L_1$找出频率为$2$项集的集合$L_2$，使用$L_2$找$L_3$，直到$k$项。

为了提高频率项集逐层产生的效率，**先验性质（Apriori property）**的性质用于压缩收缩空间。

**先验性质**：频率项集的所有非空子集一定也是频繁的。

该性质属于一类特殊的性质，**反单调性**，如果一个集合不能满足该性质，则他的所有超集也不能通过相同的测试。

**如何在算法中使用先验性质？**：考察如何使用$L_{k-1}$找出$L_k$，其中$k ≥ 2$。
1. **连接步**：为了找出$L_{k}$，将$L_{k-1}$与自身连接产生候选$k$项集的集合。该候选集的集合记为$C_{k}$。设$l_{1}$和$l_{2}$是$L_{k-1}$中的项集。记号$l_{i}[j]$表示$l_{i}$的第$j$项（例如，$l_{i}[k-2]$表示$l_{i}$的倒数第2项）。
    为了有效实现，$Apriori$算法假定事务或项集中的项按照字典排序。对于$(k-1)$项集$l_i$，排序，使得$l_{i}[1] < l_{i}[2]< \cdots < l_{i}[k-1]$。条件（$l_{1}[k-1] < l_{2}[k-1]$）是保证不产生重复。连接$l_{1}$和$l_{2}$的结果为，产生结果的项集是$ \{ l_{1}[1],l_{1}[2],\cdots,l_{1}[k-1],l_{2}[k-1] \} $。

2. **剪枝步**：$C_k$是$L_k$的超集，也就是说$C_k$的成员可以不用是频繁的，但是所有频繁的项都包含在$C_k$种。即剪枝步可以删除$C_k$中的非频繁项。

**例子如下：**
![](images/Apriori.JPG)

## 频率项集产生关联规则

即 $$ confidence(A \Rightarrow B) = P(A|B) = \frac{support\_count(A \cup B)}{support\_count(A)} $$

一旦从数据种找出频率项集，就可以由他们直接产生强关联规则（即同时满足最小置信度和最小支持度）

如上表(Table 6.1)中的数据：

$$ \{I1,I2\} \Rightarrow I5, confidence = 2/4 = 50\% $$
$$ \{I1,I5\} \Rightarrow I2, confidence = 2/2 = 100\% $$
$$ \{I2,I5\} \Rightarrow I1, confidence = 2/2 = 100\% $$
$$ I1 \Rightarrow \{I2,I5\}, confidence = 2/6 = 33\% $$
$$ I2 \Rightarrow \{I1,I5\}, confidence = 2/7 = 29\% $$
$$ I5 \Rightarrow \{I1,25\}, confidence = 2/2 = 100\% $$

第二、三和最后一项是强规则。

## 频繁模式增长（Frequent-Pattern Growth, FP-growth）提高Apriori算法的效率

此算法不产生候选选项。

如：
![](images/fp-growth.JPG)